In [1]:
import numpy as np
import os 
import h5py
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'
import glob
from natsort import natsorted
from sklearn.model_selection import train_test_split
import sigpy as sp
import sigpy.mri as mri
from subsample import *
import matplotlib.pyplot as plt
from fastmri.data import transforms, mri_data


In [2]:
kspace_path_files = glob.glob('data/raw_knee3d/*.npy')
print(kspace_path_files)

['data/raw_knee3d/2588bfa8-0c97-478c-aa5a-487cc88a590d.npy', 'data/raw_knee3d/52c2fd53-d233-4444-8bfd-7c454240d314.npy', 'data/raw_knee3d/8ad53ab7-07f9-4864-98d0-dc43145ff588.npy', 'data/raw_knee3d/b65b9167-a0d0-4295-9dd5-74641b3dd7e6.npy', 'data/raw_knee3d/cc70c22f-1ddc-4a53-9503-e33d613de321.npy', 'data/raw_knee3d/d089cbe0-48b3-4ae2-9475-53ca89ee90fe.npy', 'data/raw_knee3d/280cf3f9-3b7e-4738-84e0-f72b21aa5266.npy', 'data/raw_knee3d/ee2efe48-1e9d-480e-9364-e53db01532d4.npy', 'data/raw_knee3d/9a740e7b-8fc3-46f9-9f70-1b7bedec37e4.npy', 'data/raw_knee3d/b7d435a1-2421-48d2-946c-d1b3372f7c60.npy', 'data/raw_knee3d/7a9f028c-8667-48aa-8e08-0acf3320c8d4.npy', 'data/raw_knee3d/1b197efe-9865-43be-ac24-f237c380513e.npy', 'data/raw_knee3d/226e710b-725b-4bec-840e-bf47be2b8a44.npy', 'data/raw_knee3d/dd969854-ec56-4ccc-b7ac-ff4cd7735095.npy', 'data/raw_knee3d/54c077b2-7d68-4e77-b729-16afbccae9ac.npy', 'data/raw_knee3d/efa383b6-9446-438a-9901-1fe951653dbd.npy', 'data/raw_knee3d/ec00945c-ad90-46b7-8c3

In [3]:
def write_data_h5f(ksp, sens, save_path):
    
    with h5py.File(save_path, 'w') as hw:
        hw.create_dataset('kspace',
                          data=np.complex64(kspace)) 
        hw.create_dataset('sens', data=np.complex64(sensemap))

    return

In [ ]:
new_path = 'data/proc_knee3d'
for idx, ksp_path in enumerate(kspace_path_files):
    kspace = np.load(ksp_path)
    kspace = sp.ifft(kspace, axes=[-1]) # 

    shape_x = kspace.shape[-1]

    JsenseApp = mri.app.JsenseRecon(kspace, ksp_calib_width=20)
    sensemap = JsenseApp.run()
    del JsenseApp
    sensemap = sensemap.astype(np.complex64)
    
    sensemap = np.moveaxis(sensemap,-1,0)
    kspace = np.moveaxis(kspace,-1,0)

    print('Final Shape:',sensemap.shape, kspace.shape)
    save_name = str(idx) + '.h5'
    save_path = os.path.join(new_path, save_name)
    write_data_h5f(kspace, sensemap, save_path)
    print(save_path)

In [ ]:
with h5py.File('data/div_knee3d/Train/0.h5', 'r') as hr:
    kspace = hr['kspace'][:]
    sens = hr['sens'][:]

In [ ]:
print(kspace.shape, sens.shape)

In [ ]:
ksp_slice, sens_slice = kspace[100], sens[100]

In [ ]:
im_comb = np.sum(sp.ifft(ksp_slice,axes=(1,2)) * np.conj(sens_slice), axis=0)
print(im_comb.shape)

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(np.abs(im_comb), cmap='gray')
plt.axis('off')
plt.show()

In [4]:
from torch.utils.data import DataLoader, Dataset



In [5]:
import sigpy.mri
import numpy as np
import torch
from math import floor, ceil

class MaskFunc:
    """
    Abstract MaskFunc class for creating undersampling masks of a specified shape.
    """

    def __init__(self, accelerations):
        """
        Args:
            accelerations (List[int]): Range of acceleration rates to simulate.
        """
        self.accelerations = accelerations
        self.rng = np.random.RandomState()

    def choose_acceleration(self):
        """
        Chooses a random acceleration rate given a range.
        """
        accel_range = self.accelerations[1] - self.accelerations[0]
        acceleration = self.accelerations[0] + accel_range*self.rng.rand()
        return acceleration


In [31]:
class PoissonDiskMaskFunc(MaskFunc):
    """
    PoissonDiskMaskFunc creates a 2D Poisson disk undersampling mask.
    """
    def __init__(self, accelerations, calib_size):
        super().__init__(accelerations)
        self.calib_size = [calib_size, calib_size]

    def __call__(self, out_shape, seed=None):
        #self.rng.seed(seed)

        # Design parameters for mask
        nky = out_shape[0]
        nkz = out_shape[1]
        acceleration = self.choose_acceleration()

        # Generate undersampling mask
        mask = sigpy.mri.poisson([nky, nkz], acceleration,
                calib=self.calib_size,
                dtype=np.float32)

        return mask

In [17]:
mask_func = PoissonDiskMaskFunc(accelerations=[9,12],calib_size=20)

In [19]:
mask = mask_func((256,320))
print(mask.shape)

256 320 11.709776092175609 [20, 20]
(256, 320)


In [8]:
mask1 = mri.poisson([256,320], 11.5, [20,20])


In [22]:
import pathlib

In [47]:
class kneeData(Dataset):
    def __init__(self, root, acc=[9, 12]):
        super().__init__()
        self.examples = []
        Files = list(pathlib.Path(root).glob('*.h5'))
        for fname in sorted(Files):
            data = h5py.File(fname, 'r')
            ksp = data['kspace']

            num_slices = ksp.shape[0]
            self.examples += [(fname, slice_num)
                              for slice_num in range(num_slices)]
        self.mask_func = PoissonDiskMaskFunc(accelerations=acc,
                                             calib_size=20)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        fname, sl = self.examples[idx]
        with h5py.File(fname, 'r') as hr:
            kspace, sens = hr['kspace'][sl], hr['sens'][sl]

        im_coil = sp.ifft(kspace, axes=[1, 2])
        im_comb = np.sum(im_coil * np.conj(sens), axis=0)
        
        mask = self.mask_func(im_comb.shape)
        mask = np.expand_dims(mask, axis=0)
        masked_kspace = kspace * mask
        mask = np.expand_dims(mask, axis=-1)
        
        masked_kspace = transforms.to_tensor(masked_kspace)
        mask = transforms.to_tensor(mask)
        sens = transforms.to_tensor(sens)
        
        im_comb = transforms.to_tensor(im_comb)
                        
        return masked_kspace, mask.byte(), sens, im_comb

In [48]:
dset = kneeData(root='data/div_knee3d/Test')

In [49]:
k, m, s, i = dset[0]
print(k.shape, m.shape, s.shape, i.shape)

torch.Size([8, 256, 320, 2]) torch.Size([1, 256, 320, 1]) torch.Size([8, 256, 320, 2]) torch.Size([256, 320, 2])


In [50]:
trainloader = DataLoader(dset, batch_size=1, shuffle=False, num_workers=4)

In [51]:
tmp = iter(trainloader)

In [52]:
k, m, s, i = next(tmp)
print(k.shape, m.shape, s.shape, i.shape)

torch.Size([1, 8, 256, 320, 2]) torch.Size([1, 1, 256, 320, 1]) torch.Size([1, 8, 256, 320, 2]) torch.Size([1, 256, 320, 2])


In [ ]:
import fastmri
from fastmri.data import subsample
from fastmri.data import transforms, mri_data
from fastmri.data import SliceDataset
import pathlib

In [ ]:
def data_transform(kspace, mask, target, data_attributes, filename, slice_num, seed=None):
    # Transform the data into appropriate format
    # Here we simply mask the k-space and return the result
    kspace = transforms.to_tensor(kspace)
#    masked_kspace, mask = transforms.apply_mask(kspace, mask_func)
    
#    max_value = data_attributes["max"].astype('float32')
#    crop_size = torch.tensor([data_attributes["recon_size"][0], data_attributes["recon_size"][1]])
#    target = fastmri.rss(fastmri.complex_abs(fastmri.ifft2c(kspace)), dim=0)
    
    return kspace

In [ ]:
trainset = SliceDataset(root= pathlib.Path('data/proc_knee3d'), transform=data_transform, challenge='multicoil')


In [ ]:

print(sensemap.shape)

In [ ]:
sens_slice = sensemap[0,...,50]

In [ ]:
ksp = kspace_slice[...,50]
im_coil = sp.ifft(ksp, axes=[1,2])

In [ ]:
im_coil = sp.ifft(ksp, axes=[1,2])
im_comb = np.sum(im_coil * np.conj(sens_slice), axis=0)

In [ ]:
plt.imshow(np.abs(im_comb), cmap='gray')

In [ ]:
mask1 = mri.poisson([256,320], 12, [20,20])
plt.imshow(np.real(mask1))